# makeOBCandTIDALforcing

for MOM6 tide runs

I use Ashley Barnes' existing tidal forcings and OBCs generated from https://github.com/COSIMA/regional-mom6 to make the same structure of the obc files.

In [1]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
from xgcm import Grid
import numpy as np
import pandas as pd
import cftime
import IPython.display
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import sys, os
import warnings
warnings.simplefilter("ignore")
from dask.distributed import Client

from xhistogram.xarray import histogram


In [2]:
# open existing run to make OBC
prog = xr.open_dataset('/scratch/x77/cy8964/mom6/archive/layer-thermo-on-warm_SF/output000/prog__0002_002.nc')
prog

<xarray.Dataset> Size: 4GB
Dimensions:  (xq: 241, yh: 40, zl: 36, Time: 365, xh: 240, yq: 41, zi: 37)
Coordinates:
  * xq       (xq) float64 2kB 320.0 322.0 324.0 326.0 ... 796.0 798.0 800.0
  * yh       (yh) float64 320B 1.0 3.0 5.0 7.0 9.0 ... 71.0 73.0 75.0 77.0 79.0
  * zl       (zl) float64 288B 1.027e+03 1.027e+03 ... 1.028e+03 1.028e+03
  * Time     (Time) object 3kB 0002-01-02 00:00:00 ... 0003-01-01 00:00:00
  * xh       (xh) float64 2kB 321.0 323.0 325.0 327.0 ... 795.0 797.0 799.0
  * yq       (yq) float64 328B 0.0 2.0 4.0 6.0 8.0 ... 72.0 74.0 76.0 78.0 80.0
  * zi       (zi) float64 296B 1.027e+03 1.027e+03 ... 1.028e+03 1.028e+03
Data variables:
    u        (Time, zl, yh, xq) float32 507MB ...
    v        (Time, zl, yq, xh) float32 517MB ...
    h        (Time, zl, yh, xh) float64 1GB ...
    e        (Time, zi, yh, xh) float32 519MB ...
    temp     (Time, zl, yh, xh) float32 505MB ...
    salt     (Time, zl, yh, xh) float32 505MB ...
    tr1      (Time, zl, yh, xh) float32 505MB ...
Attributes:
    NumFilesInSet:     1
    title:             MOM Experiment
    associated_files:  areacello: ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A

In [3]:
# open tidal forcings
tu_003 = xr.open_dataset('/g/data/x77/cy8964/tidalforcing/forcing/tu_003.nc')
tz_003 = xr.open_dataset('/g/data/x77/cy8964/tidalforcing/forcing/tz_003.nc')


In [4]:
# open tidal forcings and rename stuff
tu_003_cp = tu_003.isel(ny_segment_003 = np.arange(41))
tu_003_cp['lat_segment_003'] = prog.yq.values
tu_003_cp = tu_003_cp.rename({'lon_segment_003':'lon_segment_001','lat_segment_003':'lat_segment_001',
                              'ny_segment_003':'ny_segment_001','nx_segment_003':'nx_segment_001',
                              'uamp_segment_003':'uamp_segment_001','vamp_segment_003':'vamp_segment_001',
                              'uphase_segment_003':'uphase_segment_001','vphase_segment_003':'vphase_segment_001'})
tu_003_cp

<xarray.Dataset> Size: 2kB
Dimensions:             (ny_segment_001: 41, lat_segment_001: 41, time: 1,
                         constituent: 1, nx_segment_001: 1)
Coordinates:
    lon_segment_001     (ny_segment_001) float64 328B ...
  * lat_segment_001     (lat_segment_001) float64 328B 0.0 2.0 4.0 ... 78.0 80.0
  * time                (time) datetime64[ns] 8B 2009-12-01
Dimensions without coordinates: ny_segment_001, constituent, nx_segment_001
Data variables:
    uamp_segment_001    (time, constituent, ny_segment_001, nx_segment_001) float64 328B ...
    vamp_segment_001    (time, constituent, ny_segment_001, nx_segment_001) float64 328B ...
    uphase_segment_001  (time, constituent, ny_segment_001, nx_segment_001) float64 328B ...
    vphase_segment_001  (time, constituent, ny_segment_001, nx_segment_001) float64 328B ...

In [5]:
tz_003_cp = tz_003.isel(ny_segment_003 = np.arange(41))
tz_003_cp['lat_segment_003'] = prog.yq.values
tz_003_cp = tz_003_cp.rename({'lon_segment_003':'lon_segment_001','lat_segment_003':'lat_segment_001',
                              'ny_segment_003':'ny_segment_001','nx_segment_003':'nx_segment_001',
                              'zamp_segment_003':'zamp_segment_001','zphase_segment_003':'zphase_segment_001'})


Now I can make a tu and tz for each desired idealised barotropic tide (the existing ones are from real tpxo tide model)

From my volume analysis,

$$\Delta(\text{SSH}) = U_{amp} * 32.02 \text{s}$$

So big tides should have $U_{amp} = 0.1$ms$^{-1}$ and SSH=3.202m;

Mid tides should have $U_{amp} = 0.05$ms$^{-1}$ and SSH=1.601m;

Small tides should have $U_{amp} = 0.01$ms$^{-1}$ and SSH=0.3202m;

In [7]:
#BigTides first

tu_003_cp['uamp_segment_001'] = tu_003_cp.uamp_segment_001*0+0.1
tu_003_cp['vamp_segment_001'] = tu_003_cp.vamp_segment_001*0
tu_003_cp['uphase_segment_001'] = tu_003_cp.uphase_segment_001*0
tu_003_cp['vphase_segment_001'] = tu_003_cp.vphase_segment_001*0
tu_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tu_001_u01-v0.nc')

tz_003_cp['zamp_segment_001'] = tz_003_cp.zamp_segment_001*0+3.202 #0.7 #3.5 #0.2
tz_003_cp['zphase_segment_001'] = tz_003_cp.zphase_segment_001*0 +90
tz_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tz_001_eta_32_ps90.nc')

In [8]:
#MediumTides

tu_003_cp['uamp_segment_001'] = tu_003_cp.uamp_segment_001*0+0.05
tu_003_cp['vamp_segment_001'] = tu_003_cp.vamp_segment_001*0
tu_003_cp['uphase_segment_001'] = tu_003_cp.uphase_segment_001*0
tu_003_cp['vphase_segment_001'] = tu_003_cp.vphase_segment_001*0
tu_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tu_001_u005-v0.nc')

tz_003_cp['zamp_segment_001'] = tz_003_cp.zamp_segment_001*0+1.601
tz_003_cp['zphase_segment_001'] = tz_003_cp.zphase_segment_001*0 +90
tz_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tz_001_eta_16_ps90.nc')

In [9]:
#SmallTides

tu_003_cp['uamp_segment_001'] = tu_003_cp.uamp_segment_001*0+0.01
tu_003_cp['vamp_segment_001'] = tu_003_cp.vamp_segment_001*0
tu_003_cp['uphase_segment_001'] = tu_003_cp.uphase_segment_001*0
tu_003_cp['vphase_segment_001'] = tu_003_cp.vphase_segment_001*0
tu_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tu_001_u001-v0.nc')

tz_003_cp['zamp_segment_001'] = tz_003_cp.zamp_segment_001*0+0.3202
tz_003_cp['zphase_segment_001'] = tz_003_cp.zphase_segment_001*0 +90
tz_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tz_001_eta_032_ps90.nc')

In [6]:
#GiantTides

tu_003_cp['uamp_segment_001'] = tu_003_cp.uamp_segment_001*0+0.2
tu_003_cp['vamp_segment_001'] = tu_003_cp.vamp_segment_001*0
tu_003_cp['uphase_segment_001'] = tu_003_cp.uphase_segment_001*0
tu_003_cp['vphase_segment_001'] = tu_003_cp.vphase_segment_001*0
tu_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tu_001_u02-v0.nc')

tz_003_cp['zamp_segment_001'] = tz_003_cp.zamp_segment_001*0+6.404 #0.7 #3.5 #0.2
tz_003_cp['zphase_segment_001'] = tz_003_cp.zphase_segment_001*0 +90
tz_003_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/tz_001_eta_64_ps90.nc')

Now I can input these in the MOM_override with
```
! ADD OBC AND TIDES
OBC_NUMBER_OF_SEGMENTS = 1
OBC_FREESLIP_VORTICITY = True
OBC_FREESLIP_STRAIN = True
OBC_SEGMENT_001 = "I=N,J=0:N,FLATHER,ORLANSKI,NUDGED"
OBC_SEGMENT_001_VELOCITY_NUDGING_TIMESCALES = .3, 360.0 ! inflow and outflow timescales
OBC_SEGMENT_001_DATA = "U=file:forcing/warmR22lim-forcing_obc_segment_001.nc(u),V=file:forcing/warmR22lim-forcing_obc_segment_001.nc(v),SSH=file:forcing/warmR22lim-forcing_obc_segment_001.nc(eta),TEMP=file:forcing/warmR22lim-forcing_obc_segment_001.nc(temp),SALT=file:forcing/warmR22lim-forcing_obc_segment_001.nc(salt),Uamp=file:forcing/tu_001_u01-v0.nc(uamp),Uphase=file:forcing/tu_001_u01-v0.nc(uphase),Vamp=file:forcing/tu_001_u01-v0.nc(vamp),Vphase=file:forcing/tu_001_u01-v0.nc(vphase),SSHamp=file:forcing/tz_001_eta_35_ps90.nc(zamp),SSHphase=file:forcing/tz_001_eta_35_ps90.nc(zphase)"

OBC_TIDE_N_CONSTITUENTS = 1
OBC_TIDE_CONSTITUENTS = "M2"
OBC_TIDE_REF_DATE = 1995, 1, 1


```

# Now we make an OBC forcing file

use 'iceland' which is a domain from Ashley's MOM6 regional scripts around iceland (but it doesn't matter, we just want to get the shapes and names right

In [15]:
iceland = xr.open_dataset('/g/data/x77/cy8964/tidalforcing/forcing_obc_segment_003.nc')
iceland

<xarray.Dataset> Size: 7MB
Dimensions:              (time: 4, nz_segment_003_u: 75, ny_segment_003: 465,
                          nx_segment_003: 1, nz_segment_003_v: 75,
                          nz_segment_003_salt: 75, nz_segment_003_temp: 75,
                          nz_salt_segment_003: 75, nz_temp_segment_003: 75,
                          nz_u_segment_003: 75, nz_v_segment_003: 75)
Coordinates:
  * time                 (time) object 32B 2171-01-01 00:00:00 ... 2171-01-04...
  * nz_segment_003_salt  (nz_segment_003_salt) int32 300B 0 1 2 3 ... 72 73 74
  * nz_segment_003_temp  (nz_segment_003_temp) int32 300B 0 1 2 3 ... 72 73 74
  * nz_segment_003_u     (nz_segment_003_u) int32 300B 0 1 2 3 4 ... 71 72 73 74
  * nz_segment_003_v     (nz_segment_003_v) int32 300B 0 1 2 3 4 ... 71 72 73 74
  * ny_segment_003       (ny_segment_003) int32 2kB 0 1 2 3 ... 461 462 463 464
  * nx_segment_003       (nx_segment_003) int32 4B 0
Dimensions without coordinates: nz_salt_segment_003, nz_temp_segment_003,
                                nz_u_segment_003, nz_v_segment_003
Data variables:
    u_segment_003        (time, nz_segment_003_u, ny_segment_003, nx_segment_003) float32 558kB ...
    v_segment_003        (time, nz_segment_003_v, ny_segment_003, nx_segment_003) float32 558kB ...
    eta_segment_003      (time, ny_segment_003, nx_segment_003) float32 7kB ...
    salt_segment_003     (time, nz_segment_003_salt, ny_segment_003, nx_segment_003) float32 558kB ...
    temp_segment_003     (time, nz_segment_003_temp, ny_segment_003, nx_segment_003) float32 558kB ...
    dz_salt_segment_003  (time, nz_salt_segment_003, ny_segment_003, nx_segment_003) float64 1MB ...
    dz_temp_segment_003  (time, nz_temp_segment_003, ny_segment_003, nx_segment_003) float64 1MB ...
    dz_u_segment_003     (time, nz_u_segment_003, ny_segment_003, nx_segment_003) float64 1MB ...
    dz_v_segment_003     (time, nz_v_segment_003, ny_segment_003, nx_segment_003) float64 1MB ...
    lon_segment_003      (ny_segment_003, nx_segment_003) float64 4kB ...
    lat_segment_003      (ny_segment_003, nx_segment_003) float64 4kB ...
Attributes:
    regrid_method:  bilinear

In [16]:
# open existing run to make OBC
run = 'layer-thermo-on-cold_SF'
prog = xr.open_dataset('/scratch/x77/cy8964/mom6/archive/'+run+'/output000/prog__0002_002.nc')
AveragingTimeSlice = np.arange(-60,0)

In [14]:
# set up time array for 2 years

times = []
counter = 0
for i in np.arange(1,13):
    days_in_mo = [31,28,31,30,31,30,31,31,30,31,30,31][i-1]
    for j in np.arange(1,days_in_mo+1):
        #print(cftime.DatetimeJulian(1, i, j, 12, 0, 0, 0, has_year_zero=False))
        times = np.append(times,cftime.DatetimeJulian(1, i, j, 0, 0, 0, 0, has_year_zero=False))
for i in np.arange(1,13):
    days_in_mo = [31,28,31,30,31,30,31,31,30,31,30,31][i-1]
    for j in np.arange(1,days_in_mo+1):
        #print(cftime.DatetimeJulian(1, i, j, 12, 0, 0, 0, has_year_zero=False))
        times = np.append(times,cftime.DatetimeJulian(2, i, j, 0, 0, 0, 0, has_year_zero=False))
        
    #counter =+ days_in_mo
times = np.append(times,cftime.DatetimeJulian(3, 1, 1, 0, 0, 0, 0, has_year_zero=False))
times.shape #= times[:360] # since I have only run original sponge-forced for 360 days

(731,)

In [18]:
#make iceland copy and add times and change to be segment 001 and the correct layer/domain size
#change data to be last 2 months of end of year 2 sponge-forced version with no tides
#u should be zero (wall), just v that may have barotropic circ...

iceland_cp = iceland.pad({'time':(0,727)}) #4 original, so 731 in total
iceland_cp = iceland_cp.isel(nz_segment_003_u = np.arange(36),
                             nz_segment_003_v = np.arange(36),
                             nz_segment_003_salt = np.arange(36),
                             nz_segment_003_temp = np.arange(36),
                             ny_segment_003 = np.arange(40),
                             nz_u_segment_003 = np.arange(36),
                             nz_v_segment_003 = np.arange(36),
                             nz_salt_segment_003 = np.arange(36),
                             nz_temp_segment_003 = np.arange(36))
iceland_cp['time'] = times 

iceland_cp['u_segment_003'] = iceland_cp.u_segment_003.fillna(0)*0+prog.u.sel(xq = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#.rename({'xq':'nx_segment','yh':'ny_segment','z_l':'nz_segment_003_u'})#.values
iceland_cp['v_segment_003'] = iceland_cp.v_segment_003.fillna(0)*0+prog.v.sel(xh = slice(799,801), yq = slice(0,79)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#rename({'xh':'nx_segment','yq':'ny_segment','z_l':'nz_segment_003_v'})
iceland_cp['temp_segment_003'] = iceland_cp.temp_segment_003.fillna(0)*0+prog.temp.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#.rename({'xh':'nx_segment','yh':'ny_segment','z_l':'nz_segment_003_temp'})
iceland_cp['salt_segment_003'] = iceland_cp.salt_segment_003.fillna(0)*0+ prog.salt.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#rename({'xh':'nx_segment','yh':'ny_segment','z_l':'nz_segment_003_salt'})
iceland_cp['eta_segment_003'] = iceland_cp.eta_segment_003.fillna(0)*0+prog.e.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').isel(zi = 0).fillna(0).values#rename({'xh':'nx_segment','yh':'ny_segment'})

iceland_cp['dz_u_segment_003'] = (iceland_cp.dz_u_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
iceland_cp['dz_v_segment_003'] = (iceland_cp.dz_v_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
iceland_cp['dz_temp_segment_003'] = (iceland_cp.dz_temp_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
iceland_cp['dz_salt_segment_003'] = (iceland_cp.dz_salt_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values

iceland_cp['lat_segment_003'] = iceland_cp.lat_segment_003*0+np.vstack(prog.yh.values)#rename({'xh':'nx_segment_003','yh':'ny_segment_003'})
iceland_cp['lon_segment_003'] = iceland_cp.lon_segment_003*0+80#.rename({'xh':'nx_segment_003','yh':'ny_segment_003'})

#rename to 001
iceland_cp = iceland_cp.rename({'lon_segment_003':'lon_segment_001','lat_segment_003':'lat_segment_001',
                              'ny_segment_003':'ny_segment_001','nx_segment_003':'nx_segment_001',
                              'u_segment_003':'u_segment_001','v_segment_003':'v_segment_001',
                              'temp_segment_003':'temp_segment_001','salt_segment_003':'salt_segment_001',
                              'eta_segment_003':'eta_segment_001','dz_u_segment_003':'dz_u_segment_001',
                               'dz_v_segment_003':'dz_v_segment_001','dz_temp_segment_003':'dz_temp_segment_001',
                               'dz_salt_segment_003':'dz_salt_segment_001',
                               'nz_segment_003_salt':'nz_segment_001_salt',
                               'nz_segment_003_temp':'nz_segment_001_temp',
                               'nz_segment_003_u':'nz_segment_001_u',
                               'nz_segment_003_v':'nz_segment_001_v',
                               'nz_u_segment_003':'nz_u_segment_001',
                               'nz_v_segment_003':'nz_v_segment_001',
                               'nz_temp_segment_003':'nz_temp_segment_001','nz_salt_segment_003':'nz_salt_segment_001'})


In [19]:
#save
encoding_dict = {
            "time": {
                "dtype": "double",
            }}
iceland_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/'+run+'-forcing_obc_segment_001.nc',
                    encoding=encoding_dict,
                    unlimited_dims="time",)

Repeat for the other ISOMIP+ (prescribed tidal forcing melt experiments) to get the other OBCs

In [20]:
# open existing run to make OBC
for run in ['layer-thermo-on-cold_CC','layer-thermo-on-warm_CC','layer-thermo-on-warm_SF']:
    print(run)
    prog = xr.open_dataset('/scratch/x77/cy8964/mom6/archive/'+run+'/output000/prog__0002_002.nc')
    AveragingTimeSlice = np.arange(-60,0)
    
    #make iceland copy and add times and change to be segment 001 and the correct layer/domain size
    #change data to be last 2 months of end of year 2 sponge-forced version with no tides
    #u should be zero (wall), just v that may have barotropic circ...
    
    iceland_cp = iceland.pad({'time':(0,727)}) #4 original, so 731 in total
    iceland_cp = iceland_cp.isel(nz_segment_003_u = np.arange(36),
                                 nz_segment_003_v = np.arange(36),
                                 nz_segment_003_salt = np.arange(36),
                                 nz_segment_003_temp = np.arange(36),
                                 ny_segment_003 = np.arange(40),
                                 nz_u_segment_003 = np.arange(36),
                                 nz_v_segment_003 = np.arange(36),
                                 nz_salt_segment_003 = np.arange(36),
                                 nz_temp_segment_003 = np.arange(36))
    iceland_cp['time'] = times 
    
    iceland_cp['u_segment_003'] = iceland_cp.u_segment_003.fillna(0)*0+prog.u.sel(xq = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#.rename({'xq':'nx_segment','yh':'ny_segment','z_l':'nz_segment_003_u'})#.values
    iceland_cp['v_segment_003'] = iceland_cp.v_segment_003.fillna(0)*0+prog.v.sel(xh = slice(799,801), yq = slice(0,79)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#rename({'xh':'nx_segment','yq':'ny_segment','z_l':'nz_segment_003_v'})
    iceland_cp['temp_segment_003'] = iceland_cp.temp_segment_003.fillna(0)*0+prog.temp.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#.rename({'xh':'nx_segment','yh':'ny_segment','z_l':'nz_segment_003_temp'})
    iceland_cp['salt_segment_003'] = iceland_cp.salt_segment_003.fillna(0)*0+ prog.salt.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values#rename({'xh':'nx_segment','yh':'ny_segment','z_l':'nz_segment_003_salt'})
    iceland_cp['eta_segment_003'] = iceland_cp.eta_segment_003.fillna(0)*0+prog.e.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').isel(zi = 0).fillna(0).values#rename({'xh':'nx_segment','yh':'ny_segment'})
    
    iceland_cp['dz_u_segment_003'] = (iceland_cp.dz_u_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
    iceland_cp['dz_v_segment_003'] = (iceland_cp.dz_v_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
    iceland_cp['dz_temp_segment_003'] = (iceland_cp.dz_temp_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
    iceland_cp['dz_salt_segment_003'] = (iceland_cp.dz_salt_segment_003.fillna(0)*0+1)+prog.h.sel(xh = slice(799,801)).isel(Time = AveragingTimeSlice).mean('Time').fillna(0).values
    
    iceland_cp['lat_segment_003'] = iceland_cp.lat_segment_003*0+np.vstack(prog.yh.values)#rename({'xh':'nx_segment_003','yh':'ny_segment_003'})
    iceland_cp['lon_segment_003'] = iceland_cp.lon_segment_003*0+80#.rename({'xh':'nx_segment_003','yh':'ny_segment_003'})
    
    #rename to 001
    iceland_cp = iceland_cp.rename({'lon_segment_003':'lon_segment_001','lat_segment_003':'lat_segment_001',
                                  'ny_segment_003':'ny_segment_001','nx_segment_003':'nx_segment_001',
                                  'u_segment_003':'u_segment_001','v_segment_003':'v_segment_001',
                                  'temp_segment_003':'temp_segment_001','salt_segment_003':'salt_segment_001',
                                  'eta_segment_003':'eta_segment_001','dz_u_segment_003':'dz_u_segment_001',
                                   'dz_v_segment_003':'dz_v_segment_001','dz_temp_segment_003':'dz_temp_segment_001',
                                   'dz_salt_segment_003':'dz_salt_segment_001',
                                   'nz_segment_003_salt':'nz_segment_001_salt',
                                   'nz_segment_003_temp':'nz_segment_001_temp',
                                   'nz_segment_003_u':'nz_segment_001_u',
                                   'nz_segment_003_v':'nz_segment_001_v',
                                   'nz_u_segment_003':'nz_u_segment_001',
                                   'nz_v_segment_003':'nz_v_segment_001',
                                   'nz_temp_segment_003':'nz_temp_segment_001','nz_salt_segment_003':'nz_salt_segment_001'})
    
    #save
    encoding_dict = {
                "time": {
                    "dtype": "double",
                }}
    iceland_cp.to_netcdf('/g/data/x77/cy8964/tidalforcing/forcing/'+run+'-forcing_obc_segment_001.nc',
                        encoding=encoding_dict,
                        unlimited_dims="time",)

layer-thermo-on-cold_CC
layer-thermo-on-warm_CC
layer-thermo-on-warm_SF
